In [18]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer


In [25]:
# Assert that the model works and is loaded

import torch


model_name = 'google/pegasus-pubmed'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = PegasusTokenizer.from_pretrained(model_name)

model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
assert tgt_text[0] == "California's largest electricity provider has turned off power to hundreds of thousands of customers."

SyntaxError: EOL while scanning string literal (<ipython-input-25-b302d3974e5b>, line 4)

In [26]:
## Get Dataset 

print(tgt_text[])

SyntaxError: invalid syntax (<ipython-input-26-d921dee8c6d9>, line 3)

In [11]:
## Fine Tune

In [30]:
src_text = [
    """
The coronavirus variant first identified in Brazil, known as P.1, is resistant to three of the four antibody therapies with emergency use authorization in the United States, according to a laboratory study. In test-tube experiments, researchers exposed the P.1 variant to various monoclonal antibodies, including the four currently being used to treat U.S. COVID-19 patients - imdevimab and casirivimab from Regneron Pharmaceuticals, and bamlanivimab and etesevimab from Eli Lilly and Co. Only imdevimab retained any potency, researchers found. The neutralizing ability of the other three were "markedly or completely abolished," according to a peer reviewed report available on bioRxiv and provisionally accepted by the journal Cell Host & Microbe. The researchers also exposed P.1 to plasma from COVID-19 survivors and blood from recipients of vaccines from Pfizer/BioNTech or Moderna. Compared to their effects against the original version of the coronavirus, the plasma and the vaccine-induced antibodies were less effective at neutralizing P.1. In earlier studies, however, they were even less effective against the B.1.351 variant first identified in South Africa. This suggests that the Brazil variant might not pose as great a threat of reinfection or decreased vaccine protection as the South Africa variant, said coauthor David Ho from Columbia University. Real-world evidence is needed to confirm the lab results, he said. (bit.ly/2Qgv4j1)
    """
]
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [31]:
print(tgt_text)

['the recent outbreak of severe acute respiratory syndrome ( sars ) caused by a coronavirus variant ( known as as.1,1 ) in saudi arabia led to the identification of a new variant ( known as.351 ) of the virus. <n> the resistance of the variant to several anti - coronavirus therapies has been reported. <n> however, no reinfection of the variant has been reported. <n> the resistance of the variant to anti - coronavirus therapies has been confirmed by laboratory studies. <n> the resistance of the variant to anti - coronavirus therapies, however, has not been confirmed by other laboratory studies. <n> the resistance of the variant to anti - coronavirus therapies, however, has been confirmed by laboratory studies. <n> the resistance of the variant to anti - coronavirus therapies, however, has not been confirmed by other laboratory studies. <n> the resistance of the variant to anti - coronavirus therapies, however, has been confirmed by laboratory studies. <n> the resistance of the variant t

In [32]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [64]:
article = """
Ontario, Canada’s most populous province, has extended and expanded its stay-at-home orders and given the police sweeping new powers to enforce the mandates in an effort to curb rising case numbers and hospitalizations.

The police will now be able to stop and question people, including those in vehicles, to make sure that their trips outside home are essential. Residents, who are not permitted to gather with members outside their household with some exceptions, could face fines if they do not comply with the new orders. The Ontario government is also expected to restrict travel between Manitoba and Quebec, and the police will set up checkpoints at the provincial borders.

The new measures come amid a sluggish vaccine campaign in Canada and the latest virus wave there, driven largely in Ontario by the highly contagious B.1.1.7 variant first detected in Britain. The increasing case numbers have strained the capacity of intensive care units in many parts of the country.

Canada has inched ahead of the United States in new daily coronavirus cases per capita, and officials warned that worse is to come. By Friday, hospitalizations were up by 22 percent; I.C.U. admissions rose by 34 percent; and each day, 41 people died from Covid-19, a 38 percent increase from the previous week.

“It now falls on me to do what is necessary,” said Doug Ford, the province’s premier, who announced on Friday that he was restricting border crossings with Manitoba and Quebec, as well as shuttering playgrounds, golf courses, basketball courts and other outdoor sports facilities, along with a variety of other measures.

The latest measures have been met with criticism, including from public health experts, the mayor of Toronto and several police departments, including the Toronto Police Service, which said on Twitter that it would “not be doing random stops of people or cars.” There were also concerns that asking the police to impose such measures could result in racial profiling.

“I know you are all sick and tired of Covid-19,” Prime Minister Justin Trudeau said on Friday as he urged Canadians to follow their provinces’ rules. “We all just want to be done with this.”

Emergency rooms, particularly in Ontario, are reaching their breaking points, as are intensive care units. In a bid to ease the strain, children’s hospitals in both Ottawa and Toronto opened their I.C.U. beds to adults.

Many factors are behind the increasing numbers. Among them is the arrival of more infectious variants of the virus. An outbreak of P.1, the variant first found in Brazil, spread throughout British Columbia and then into Alberta. Manitoba discovered its first case of the variant this week.

And the vaccine campaign has taken a hit: Moderna is cutting deliveries of its vaccine shipment to Canada and other countries, and the Johnson & Johnson vaccine, which has yet to arrive in Canada, has come under safety scrutiny.

Pfizer will sell Canada an additional eight million doses of the vaccine it has developed with BioNTech, half of which will arrive next month, and all of which will arrive by the end of July.

Many Canadians are frustrated as they see higher vaccination rates in Britain and the United States.

Canada’s vaccination strategy has been to delay second doses to allow more residents to gain the protection from at least one shot. About 2 percent of Canadians are fully vaccinated compared with 25 percent of Americans, and 19 percent have received at least one dose compared with 39 percent in the United States. Yet the scheduled increases in vaccine shipments — the Moderna slip up aside — should help Canada catch up over the next few weeks.
"""
print(len(article))

3664


In [68]:
# encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("TLDR: " + article, return_tensors="pt", max_length=512, truncation=True)

In [70]:
# generate the summarization output
outputs = model.generate(
    inputs, 
    max_length=400, 
    min_length=100, 
    length_penalty=1.5, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,  9163,  4285,     7,  1049,    18,   144,    18,  5515,  5022,
            11,  1527,  2095,   126, 11552,    12, 13321, 12871,     7,     3,
             5,  2095,    56,   230,    36,     3,   179,    12,  1190,    11,
           822,   151,     6,   379,   273,    16,  3203,     6,    12,   143,
           417,    24,    70,  6846,  1067,   234,    33,  1832,     3,     5,
             8,   126,  3629,   369, 18905,     3,     9,     3,     7,    40,
         13917,  1273, 12956,  2066,    16, 19343,    11,     8,  1251,  4301,
           106,     9, 18095,  6772,     3,     5,  2833,  1707,     7,   130,
            95,    57,  1630,  1093,   117,    27,     5,   254,     5,  1265,
             5,  7209,     7,  4659,    57,  6154,  1093,     3,     5,   284,
           239,     6,  8798,   151,  3977,    45,   638,  6961,  4481,     6,
             3,     9,  6654,  1093,     1]])
<pad> Ontario extends stay-at-home orders and gives police new powers to enforce mand